## ✫faissによる近似近傍探索処理

In [1]:
# !pip install faiss-gpu

In [2]:
# GPUによる処理を前提
import faiss

res = faiss.StandardGpuResources()
dim = 384
nlist = 256
M = 32
nbits = 8
metric = faiss.METRIC_L2
ivfpq_config = faiss.GpuIndexIVFPQConfig()
ivfpq_config.usePrecomputedTables = True
index = faiss.GpuIndexIVFPQ(res, dim, nlist, M, nbits, metric, ivfpq_config)

In [3]:
# 前のステップまでに作成した（文章ID, 文章ベクトル）を配列としたリストをロードする
import pickle

with open('livedoor_vec_list.pkl', 'rb') as f:
    livedoor_vec_list = pickle.load(f)

In [4]:
len(livedoor_vec_list)
# 7376

7376

In [5]:
# faissによる学習データの準備
import numpy as np
import random
assert not index.is_trained
train_data = [v for _, v in livedoor_vec_list]
train_data = np.array(train_data, dtype=np.float32)

In [6]:
train_data.shape
# (7376, 384)

(7376, 384)

In [7]:
train_data = np.concatenate([train_data, train_data])

In [8]:
index.train(train_data)
assert index.is_trained

In [9]:
batch_size = 10000
for i in range(0, len(livedoor_vec_list), batch_size):
    input_vecs = []
    input_ids = []
    for item_id, vec in livedoor_vec_list[i:i+batch_size]:
        input_vecs.append(vec)
        input_ids.append(item_id)
    input_vecs = np.array(input_vecs, dtype=np.float32)
    input_ids = np.array(input_ids, dtype=np.int64)
    index.add_with_ids(input_vecs, input_ids)

# 学習済みインデックスの保存（GPUからCPUへ戻す）
converted_index_for_cpu = faiss.index_gpu_to_cpu(index)
faiss.write_index(converted_index_for_cpu, 'livedoor.index')

In [10]:
type(index)

faiss.swigfaiss.GpuIndexIVFPQ

### 本番利用時には保存されたindexからロードする処理から開始

In [11]:
import faiss
cpu_index = faiss.read_index('livedoor.index')
index = faiss.index_cpu_to_all_gpus(cpu_index)

In [12]:
# 文章ベクトル → 文章本文の辞書
import pickle

with open('livedoor_vec_to_text_dict.pkl', 'rb') as f:
    livedoor_vec_to_text_dict = pickle.load(f)

In [13]:
# 文章ベクトルリスト
import pickle
with open('livedoor_vec_list.pkl', 'rb') as f:
    livedoor_vec_list = pickle.load(f)

In [14]:
# 文章本文 → 文章ラベルの辞書
import pickle

with open('livedoor_text_to_label_dict.pkl', 'rb') as f:
    livedoor_text_to_label_dict = pickle.load(f)

### 検索用のベクトル変換

In [15]:
import pandas as pd
df = pd.read_csv('./triplet_test.tsv', sep='\t')
df.head(3)
test_text = str(df['anchor'][45])

In [16]:
print(test_text)

韓国を旅した時に、是非味わいたいのが韓国の伝統茶。トウモロコシ、柚子、ナツメ、生姜、五味子といった果物や穀物が素材になっているので、それぞれが疲労回復やストレス解消などの症状にやさしく効きます。韓国では昔から飲まれてきたもので、日本人に馴染みのある茶葉を使ったお茶よりも、ポピュラーなお茶だと言えるでしょう。韓国の伝統茶の種類は実に豊富ですが、今でもよく飲まれている代表的なお茶をいくつかご紹介します。最近は気温が上がってきているので、氷を入れて冷たくして飲むのめば体の熱がスッと下がります。【五味子（オミジャ）茶】甘味、酸味、苦味、辛味、塩味の5つの味を感じるという朝鮮五味子の実を、砂糖などで甘く煮たもの。アセロラのような酸味のある味なので、夏の暑い日に冷たくして飲むと美味しい。五味子は長引く咳の緩和、強壮作用がある。【ナツメ（テチュ）茶】ナツメの実に蜂蜜や砂糖を加えて甘く煮たお茶。ほんのりと甘く苦みは無いので、伝統茶が始めての人でも飲みやすいお茶。貧血や神経衰弱、美肌に効果がある。【柚子（ユジャ）茶】柚子を蜂蜜に漬けたお茶。お湯に溶かせばホットレモネートのような味になるので、飲みやすく日本人にも人気。ビタミンCが豊富に含まれているので、風邪予防にも。【生姜（センガン）茶】煎じた生姜に蜂蜜や砂糖を入れて甘くしたお茶。生姜の辛さを強く感じるが、飲んだ後は体がじんわりと温まる。免疫力を高め、のどの痛みに効く。【雙和（サンファ）茶】桂皮・甘草・熟地黄・山芍薬・トウキ・センキュウなど９種類の漢方を煎じ、砂糖などで甘くしたお茶。甘く飲みやすいが漢方特有の香りも強い。疲労回復のほか、疲れによる風邪に効果的。このほかにも、「花梨（モグァ）茶」「菊花（クッカ）茶」「桂皮（ケピ）茶」など、さまざまな種類があり、何を飲むか毎回迷ってしまうほどです。最近では、便利なスティックタイプも登場しているので、コップに粉を入れてお湯か水を注げば、いつでもどこでも手軽に伝統茶を楽しむことができます。仕事の疲れが取れない、のどが痛い、美肌のため…など、その日の体調に応じてお茶を選べばきっと心身ともにリラックスできるでしょう。（文：林由美）■まだまだあります！耳寄り韓国ビューティニュース・シートマスクは美肌への近道！？集中ケアで肌にごほう美を（2010年06月25日）・［穫れたて現地情報］BBクリームは夏の

In [17]:
import sentence_transformers
import numpy as np
from sentence_transformers import SentenceTransformer

sbert = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
vector = sbert.encode(test_text)
# queries = np.array([vector], dtype=np.float32)
# queries.shape
# (1, 384)

In [18]:
import faiss
import numpy as np

vecs = np.array([vector], dtype=np.float32)
d, i = index.search(vecs, 10) # 近傍ベクトルを10個とってくる
d = d[0]
i = i[0]
for item_id, distance in zip(i, d):
    # 文章IDと距離を表示
    print('文章ID:', item_id, '距離:', distance)
    n = np.array(livedoor_vec_list[item_id][1], dtype=np.float32)
    n = tuple(n)
    
    # 文章IDから文章を参照
    print(livedoor_vec_to_text_dict[n])
    print(livedoor_text_to_label_dict[livedoor_vec_to_text_dict[n]])
    print('---')

文章ID: 5421 距離: 4.289021
韓国を旅した時に、是非味わいたいのが韓国の伝統茶。トウモロコシ、柚子、ナツメ、生姜、五味子といった果物や穀物が素材になっているので、それぞれが疲労回復やストレス解消などの症状にやさしく効きます。韓国では昔から飲まれてきたもので、日本人に馴染みのある茶葉を使ったお茶よりも、ポピュラーなお茶だと言えるでしょう。韓国の伝統茶の種類は実に豊富ですが、今でもよく飲まれている代表的なお茶をいくつかご紹介します。最近は気温が上がってきているので、氷を入れて冷たくして飲むのめば体の熱がスッと下がります。【五味子（オミジャ）茶】甘味、酸味、苦味、辛味、塩味の5つの味を感じるという朝鮮五味子の実を、砂糖などで甘く煮たもの。アセロラのような酸味のある味なので、夏の暑い日に冷たくして飲むと美味しい。五味子は長引く咳の緩和、強壮作用がある。【ナツメ（テチュ）茶】ナツメの実に蜂蜜や砂糖を加えて甘く煮たお茶。ほんのりと甘く苦みは無いので、伝統茶が始めての人でも飲みやすいお茶。貧血や神経衰弱、美肌に効果がある。【柚子（ユジャ）茶】柚子を蜂蜜に漬けたお茶。お湯に溶かせばホットレモネートのような味になるので、飲みやすく日本人にも人気。ビタミンCが豊富に含まれているので、風邪予防にも。【生姜（センガン）茶】煎じた生姜に蜂蜜や砂糖を入れて甘くしたお茶。生姜の辛さを強く感じるが、飲んだ後は体がじんわりと温まる。免疫力を高め、のどの痛みに効く。【雙和（サンファ）茶】桂皮・甘草・熟地黄・山芍薬・トウキ・センキュウなど９種類の漢方を煎じ、砂糖などで甘くしたお茶。甘く飲みやすいが漢方特有の香りも強い。疲労回復のほか、疲れによる風邪に効果的。このほかにも、「花梨（モグァ）茶」「菊花（クッカ）茶」「桂皮（ケピ）茶」など、さまざまな種類があり、何を飲むか毎回迷ってしまうほどです。最近では、便利なスティックタイプも登場しているので、コップに粉を入れてお湯か水を注げば、いつでもどこでも手軽に伝統茶を楽しむことができます。仕事の疲れが取れない、のどが痛い、美肌のため…など、その日の体調に応じてお茶を選べばきっと心身ともにリラックスできるでしょう。（文：林由美）■まだまだあります！耳寄り韓国ビューティニュース・シートマスクは美肌への近道！？集中ケアで肌にごほう美を（2010年06月